In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import matplotlib.pyplot as plt

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim

import warnings
warnings.filterwarnings('ignore')
mpd.show_versions()

In [ ]:
# Read the CSV file and store it in a Pandas DataFrame
df = pd.read_csv("../Data/OpenSky/states_2020-05-25-00.csv", delimiter=',')
df

In [ ]:
df = df[df[['icao24', 'lat', 'lon','velocity','vertrate']].notnull().all(1)]

In [ ]:
s= gpd.GeoSeries(gpd.points_from_xy(df.lat, df.lon))

In [ ]:
s= s[~(s.is_empty | s.isna())]

In [ ]:
s

In [ ]:
df

In [ ]:

df.time = pd.to_datetime(df['time'], unit='s')


In [ ]:

df

In [ ]:
filtered_df = df[df[['icao24', 'lat', 'lon','velocity','vertrate']].notnull().all(1)]

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=s)


In [ ]:
gdf

In [ ]:
traj_collection = mpd.TrajectoryCollection(gdf, 'icao24', t='time')
gap = timedelta(minutes=5)


In [ ]:
print(traj_collection)

In [ ]:
s= gpd.GeoDataFrame(gpd.points_from_xy(df.lat, df.lon))

In [ ]:
split = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=15))


In [ ]:

split.add_speed(overwrite=True)

cleaned = mpd.OutlierCleaner(split).clean({'speed': 1})


In [ ]:
smoothed = mpd.KalmanSmootherCV(cleaned).smooth(process_noise_std=0.1, measurement_noise_std=10)

In [ ]:
print(smoothed)

In [ ]:
out = smoothed.to_point_gdf()


In [ ]:
output = out.to_csv('Output/aisdk-2022-11-18PointsMP.csv')
